<a href="https://colab.research.google.com/github/AliMousa11/computer-vision/blob/main/EyeDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image
import numpy as np
import math

In [4]:
img = Image.open('Image 2.jpeg').convert('RGB')
img_array = np.array(img)

In [5]:
# len(img_array)
len(img_array[1])

1024

In [7]:
def calculateSumImage(image):
  sum_image = np.zeros_like(image, dtype=np.int64)

  for i in range(image.shape[0]):
      for j in range(image.shape[1]):
          if j == 0:
              sum_image[i, j] = image[i, j]
          else:
              sum_image[i, j] = image[i, j] + sum_image[i, j-1]
  return sum_image

sum_image=calculateSumImage(img_array)
sum_image

array([[[   203,    204,    206],
        [   406,    408,    412],
        [   609,    612,    618],
        ...,
        [212595, 212499, 211927],
        [212797, 212701, 212127],
        [212999, 212903, 212327]],

       [[   203,    204,    206],
        [   406,    408,    412],
        [   609,    612,    618],
        ...,
        [210799, 210679, 210075],
        [211001, 210881, 210275],
        [211203, 211083, 210475]],

       [[   203,    204,    206],
        [   406,    408,    412],
        [   609,    612,    618],
        ...,
        [209042, 208866, 208254],
        [209244, 209068, 208454],
        [209446, 209270, 208654]],

       ...,

       [[   218,    219,    223],
        [   436,    438,    446],
        [   654,    657,    669],
        ...,
        [189041, 170272, 167251],
        [189267, 170498, 167479],
        [189493, 170724, 167707]],

       [[   217,    218,    222],
        [   434,    436,    444],
        [   651,    654,    666],
        .

In [8]:
def calculateIntegralImage(sum_image):
  integral_image = np.zeros_like(sum_image, dtype=np.int64)

  for i in range(sum_image.shape[0]):
      for j in range(sum_image.shape[1]):
          if i == 0:
              integral_image[i, j] = sum_image[i, j]
          else:
              integral_image[i, j] = sum_image[i, j] + integral_image[i-1, j]
  return integral_image
  integral_image=calculateIntegralImage(sum_image)
  integral_image

In [9]:
def calculateLocalSum(integral_image,x_0,y_0,x_1,y_1):
  local_sum=integral_image[x_1,y_1]-integral_image[x_0-1,y_1]-integral_image[x_1,y_0-1]+ integral_image[x_0-1,y_0-1]
  return local_sum


In [10]:
def detectEye(integral_image,kernel_width):
  kernel_height=0.15*kernel_width
  P1 = (-0.5*kernel_width,-0.5*kernel_height)
  P2 = (-0.05*kernel_width,0)
  P3 = (-0.5*kernel_width,0)
  P4 = (-0.05*kernel_width,0.5*kernel_height)
  P5 = (0.05*kernel_width,-0.5*kernel_height)
  P6 = (0.5*kernel_width,0)
  P7 = (0.05*kernel_width,0)
  P8 = (0.5*kernel_width,0.5*kernel_height)
  P9 = (-0.325*kernel_width,0.833*kernel_height)
  P10 = (-0.225*kernel_width,2*kernel_height)
  P11 = (-0.1*kernel_width,0.833*kernel_height)
  P12 = (0.1*kernel_width,2*kernel_height)
  P13 = (0.225*kernel_width,0.833*kernel_height)
  P14 = (0.325*kernel_width,2*kernel_height)
  rectangles = [
    (P1, P2,  1),  # LS1 (White, weight +1)
    (P3, P4, -1),  # LS2 (Black, weight -1)
    (P5, P6,  1),  # LS3 (White, weight +1)
    (P7, P8, -1),  # LS4 (Black, weight -1)
    (P9, P10, -1), # LS5 (Black, weight -1)
    (P11, P12, 1), # LS6 (White, weight +1)
    (P13, P14, -1) # LS7 (Black, weight -1)
  ]
  max_score = -math.inf
  max_position = (0, 0)
  height, width = integral_image.shape
  for j in range(height):
    for i in range(width):

        current_total_score = 0

        for (P_tl_relative, P_br_relative, weight) in rectangles:
            # Top-left corner
            tl_abs_x = int(i + P_tl_relative[0])
            tl_abs_y = int(j + P_tl_relative[1])
            # Bottom-right corner
            br_abs_x = int(i + P_br_relative[0])
            br_abs_y = int(j + P_br_relative[1])

            if tl_abs_x < 0  or tl_abs_y < 0 or br_abs_x >= width or br_abs_y >= height:
                continue
            local_sum = CalculateLocalSum(integral_image, (tl_abs_x, tl_abs_y), (br_abs_x, br_abs_y))
            current_total_score += local_sum * weight

        if current_total_score > max_score:
            max_score = current_total_score
            max_position = (i, j)

  return max_position